# Résumé automatique (FR) — Modèle perso from scratch

Objectif :
- Construire une IA de synthèse (texte → résumé) en français
- Tokenisation standard via SentencePiece (BPE/Unigram)
- Modèle perso : Transformer Encoder–Decoder (from scratch)
- Entraînement supervisé sur (text, summary)
- Évaluation : ROUGE + exemples qualitatifs

In [22]:
import os, math, random, time
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## Device, seeds, hyperparams (Markdown + Code)

On fixe un environnement reproductible (seed), on choisit le device (GPU si dispo),
et on définit des hyperparamètres "MVP" qui tournent sans exploser la VRAM.
Le modèle from scratch doit rester petit (d_model=256) sinon il apprend lentement.

In [23]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
# Chemins
TRAIN_CSV = "./fr_train.csv"
TEST_CSV = "./fr_test.csv"

# Tokenisation / séquences
MAX_IN_TOKENS = 512
MAX_OUT_TOKENS = 160
VOCAB_SIZE = 16000

# Modèle Transformer mini
D_MODEL = 256
N_HEADS = 4
ENC_LAYERS = 4
DEC_LAYERS = 4
D_FF = 1024
DROPOUT = 0.15  # ↑ régularise mieux

# Training
BATCH_SIZE = 16  # ↑ meilleure stabilité
LR = 5e-4  # ↓ plus stable (3e-4 trop élevé)
EPOCHS = 12  # ↓ suffisant (50 était trop long)
LABEL_SMOOTH = 0.1
GRAD_CLIP = 1.0

## Construire le corpus pour SentencePiece (Markdown + Code)

SentencePiece nécessite un corpus texte brut pour apprendre un vocabulaire subword.
Nous construisons donc un fichier intermédiaire `spm_corpus.txt` à partir du jeu
d'entraînement, en y plaçant les textes d'entrée et les résumés de référence.

In [ ]:
import pandas as pd
import os

# chemins (à adapter si besoin)
TRAIN_CSV = "./fr_train.csv"   # ton vrai fichier
corpus_path = "spm_corpus.txt"    # sera CRÉÉ ici

# charge le train
train_df = pd.read_csv(TRAIN_CSV)

# LIMITE LE DATASET À 1000 ROWS POUR ACCÉLÉRER LE TRAITEMENT
train_df = train_df.head(5000)

# construit le corpus texte pour SentencePiece
with open(corpus_path, "w", encoding="utf-8") as f:
    for _, row in train_df.iterrows():
        # entrée : titre + texte
        if "title" in train_df.columns:
            src = f"titre: {row['title']} texte: {row['text']}"
        else:
            src = row["text"]

        # cible : résumé
        tgt = row["summary"]

        # SentencePiece veut une phrase par ligne
        f.write(str(src).replace("\n", " ") + "\n")
        f.write(str(tgt).replace("\n", " ") + "\n")

print("✅ spm_corpus.txt créé")
print("Taille (lignes):", sum(1 for _ in open(corpus_path, encoding="utf-8")))

On entraîne un tokenizer **standard** (SentencePiece) sur `spm_corpus.txt`.  
Pourquoi : un modèle from scratch a besoin d'une tokenisation robuste (subwords) pour gérer vocabulaire, accents, mots rares.

Sorties :
- `spm_fr.model` : le tokenizer à charger dans tout le projet
- `spm_fr.vocab` : vocabulaire lisible (debug + rapport)


In [26]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Si besoin: !pip -q install sentencepiece
import os
import sentencepiece as spm

CORPUS_PATH = "spm_corpus.txt"
MODEL_PREFIX = "spm_fr"  # => spm_fr.model + spm_fr.vocab

VOCAB_SIZE = 16000

if not os.path.exists(CORPUS_PATH):
    raise FileNotFoundError(f"Je ne trouve pas {CORPUS_PATH}")

if not os.path.exists(MODEL_PREFIX + ".model"):
    spm.SentencePieceTrainer.train(
        input=CORPUS_PATH,
        model_prefix=MODEL_PREFIX,
        vocab_size=VOCAB_SIZE,
        model_type="unigram",      # "bpe" marche aussi
        character_coverage=1.0,    # français
        pad_id=0, unk_id=1, bos_id=2, eos_id=3
    )
    print("✅ Tokenizer entraîné:", MODEL_PREFIX + ".model")
else:
    print("Tokenizer déjà présent:", MODEL_PREFIX + ".model")


Tokenizer déjà présent: spm_fr.model


## Charger le tokenizer et valider encode/decode

In [28]:
sp = spm.SentencePieceProcessor()
sp.load(MODEL_PREFIX + ".model")

print("Vocab size:", sp.get_piece_size())

txt = "Le PSG a gagné, mais la météo était bizarre à Colombes."
ids = sp.encode_as_ids(txt)
print("ids:", ids[:30])
print("decode:", sp.decode_ids(ids))

Vocab size: 16000
ids: [38, 7029, 22, 2187, 4, 87, 9, 5330, 139, 2306, 4567, 233, 12, 11871, 967, 8, 7]
decode: Le PSG a gagné, mais la météo était bizarre à Colombes.


## Charger les CSV train/test + vérifier les colonnes

On charge les datasets. On attend au minimum `text` et `summary`.
Optionnel mais utile : `title` (on l'injecte dans l'entrée pour aider from scratch).

In [29]:
import pandas as pd

train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

print("train:", train_df.shape)
print("test :", test_df.shape)
print("cols :", list(train_df.columns))

required = {"text", "summary"}
missing = required - set(train_df.columns)
assert not missing, f"Colonnes manquantes: {missing}"

train_df.head(2)

train: (392902, 6)
test : (15828, 6)
cols : ['text', 'summary', 'topic', 'url', 'title', 'date']


,text,summary,topic,url,title,date
0,"Jean-Jacques Schuhl, Gilles Leroy, Christian G...","Jean-Jacques Schuhl, Gilles Leroy, Christian G...",livres,https://www.lemonde.fr/livres/article/2010/01/...,La rentrée littéraire promet un programme de b...,01/01/2010
1,Une semaine après l'attaque terroriste manquée...,Cette demande intervient une semaine après l'a...,proche-orient,https://www.lemonde.fr/proche-orient/article/2...,Gordon Brown appelle à une réunion internation...,01/01/2010


## Vérifier les longueurs en tokens



On mesure p50/p90/p95 des longueurs en tokens sur un échantillon.
Pourquoi : confirmer que MAX_IN_TOKENS/MAX_OUT_TOKENS ne tronquent pas trop et ne gaspillent pas la VRAM.


In [30]:
import numpy as np

def make_src(row):
    if "title" in row.index and pd.notna(row["title"]):
        return f"titre: {row['title']} texte: {row['text']}"
    return f"texte: {row['text']}"

def token_len(s: str) -> int:
    return len(sp.encode_as_ids(str(s)))

sample = train_df.sample(min(5000, len(train_df)), random_state=SEED)

in_lens = sample.apply(lambda r: token_len(make_src(r)), axis=1).to_numpy()
out_lens = sample["summary"].apply(token_len).to_numpy()

print("IN  tokens p50/p90/p95:", np.percentile(in_lens, [50,90,95]).astype(int))
print("OUT tokens p50/p90/p95:", np.percentile(out_lens,[50,90,95]).astype(int))


IN  tokens p50/p90/p95: [ 699 1471 1825]
OUT tokens p50/p90/p95: [36 53 60]


## Dataset + batching (padding, masks, teacher forcing)



On convertit chaque exemple en séquences d'IDs et on pad pour faire des batches GPU.
On prépare aussi `dec_in` (summary décalé) pour teacher forcing :
- entrée du décodeur : [BOS] + y[:-1]
- labels : y
Pourquoi : le décodeur apprend à prédire le prochain token.


In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass

PAD, UNK, BOS, EOS = 0, 1, 2, 3

def encode_src(row):
    src = make_src(row)
    ids = [BOS] + sp.encode_as_ids(src)[:MAX_IN_TOKENS-2] + [EOS]
    return ids

def encode_tgt(row):
    tgt = str(row["summary"])
    ids = [BOS] + sp.encode_as_ids(tgt)[:MAX_OUT_TOKENS-2] + [EOS]
    return ids

def pad_to(ids, max_len, pad_id=PAD):
    return ids + [pad_id]*(max_len-len(ids)) if len(ids) < max_len else ids[:max_len]

class SumDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        return encode_src(row), encode_tgt(row)

@dataclass
class Batch:
    src_ids: torch.Tensor
    dec_in: torch.Tensor
    labels: torch.Tensor

def collate_fn(batch):
    xs, ys = zip(*batch)
    xs = [pad_to(x, MAX_IN_TOKENS) for x in xs]
    ys = [pad_to(y, MAX_OUT_TOKENS) for y in ys]

    src_ids = torch.tensor(xs, dtype=torch.long)
    labels  = torch.tensor(ys, dtype=torch.long)

    dec_in = labels.clone()
    dec_in[:, 1:] = labels[:, :-1]
    dec_in[:, 0] = BOS

    return Batch(src_ids=src_ids, dec_in=dec_in, labels=labels)

## Split train/val + DataLoaders



On garde une validation pour suivre l'apprentissage et sauvegarder le meilleur modèle.
Pourquoi : from scratch peut sur-apprendre ou diverger, donc on checkpoint sur val_loss.

In [32]:
VAL_CSV = "./fr_validation.csv"   # <-- adapte le nom (ex: fr_valid.csv, validation.csv...)

val_df = pd.read_csv(VAL_CSV)

print("train:", train_df.shape)
print("val  :", val_df.shape)
print("test :", test_df.shape)

# sanity check colonnes
required = {"text", "summary"}
for name, df in [("train", train_df), ("val", val_df), ("test", test_df)]:
    missing = required - set(df.columns)
    assert not missing, f"Colonnes manquantes dans {name}: {missing}"

train_loader = DataLoader(SumDataset(train_df), batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(SumDataset(val_df),   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(SumDataset(test_df),  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

len(train_loader), len(val_loader), len(test_loader)

train: (392902, 6)
val  : (16059, 6)
test : (15828, 6)


(24557, 1004, 990)

## Masque causal (le décodeur ne voit pas le futur)



Le décodeur génère token par token. Il ne doit pas accéder aux tokens futurs.
On crée donc un masque triangulaire (causal) pour le self-attention du décodeur.

In [33]:
def causal_mask(T: int, device):
    # True = autorisé ; PyTorch attend souvent True=mask => on inversera dans le forward
    return torch.tril(torch.ones((T, T), dtype=torch.bool, device=device))

## Modèle perso : Transformer Encoder–Decoder (mini)



Architecture moderne du résumé abstractive :
- embeddings appris + positional encoding
- encoder : self-attention
- decoder : masked self-attention + cross-attention
Pourquoi : c'est la base des modèles de synthèse, mais ici entraînée from scratch.


In [34]:
import math
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=4096):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TransformerSummarizer(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, D_MODEL, padding_idx=PAD)
        self.pos = PositionalEncoding(D_MODEL, DROPOUT)

        self.tf = nn.Transformer(
            d_model=D_MODEL,
            nhead=N_HEADS,
            num_encoder_layers=ENC_LAYERS,
            num_decoder_layers=DEC_LAYERS,
            dim_feedforward=D_FF,
            dropout=DROPOUT,
            batch_first=True,
            norm_first=True
        )
        self.lm_head = nn.Linear(D_MODEL, vocab_size)

    def forward(self, src_ids, dec_in):
        B, S = src_ids.shape
        _, T = dec_in.shape

        src_kpm = (src_ids == PAD)  # True where pad
        tgt_kpm = (dec_in  == PAD)

        src = self.pos(self.emb(src_ids))
        tgt = self.pos(self.emb(dec_in))

        cm = causal_mask(T, src_ids.device)  # True allowed
        tgt_mask = ~cm                       # PyTorch: True = masked

        h = self.tf(
            src, tgt,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_kpm,
            tgt_key_padding_mask=tgt_kpm,
            memory_key_padding_mask=src_kpm
        )
        return self.lm_head(h)  # (B,T,V)


## Loss avec label smoothing




Label smoothing stabilise l'entraînement from scratch (moins d'overconfidence).
On ignore PAD pour ne pas apprendre sur du vide.

In [35]:
def label_smoothed_ce(logits, target, eps=0.1, ignore_index=PAD):
    B, T, V = logits.shape
    log_probs = F.log_softmax(logits, dim=-1)

    nll = F.nll_loss(
        log_probs.view(B*T, V),
        target.view(B*T),
        reduction="none",
        ignore_index=ignore_index
    ).view(B, T)

    smooth = -log_probs.mean(dim=-1)

    pad_mask = (target == ignore_index)
    nll = nll.masked_fill(pad_mask, 0.0)
    smooth = smooth.masked_fill(pad_mask, 0.0)

    denom = (~pad_mask).sum().clamp(min=1)
    return ((1-eps)*nll + eps*smooth).sum() / denom


## Initialiser modèle + optimiseur



On instancie le modèle, on le met sur GPU si dispo, et on prépare AdamW.
On affiche aussi le nombre de paramètres (utile dans le rapport).

In [36]:
model = TransformerSummarizer(vocab_size=sp.get_piece_size()).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

n_params = sum(p.numel() for p in model.parameters())
print("params:", n_params, f"({n_params/1e6:.2f} M)")

params: 15581824 (15.58 M)


c:\Users\aurel\anaconda3\envs\torch-gpu\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


## Entraînement + checkpoint du meilleur modèle


Boucle seq2seq :
- forward (teacher forcing via dec_in)
- loss
- backward
- clip gradients
- val_loss
Pourquoi : garder le meilleur modèle pour l'inférence.

In [37]:
import time
import torch

# Réglages: combien de batches max par "epoch"
MAX_TRAIN_BATCHES = 1000  # ↑ plus de données par epoch
MAX_VAL_BATCHES   = 300    # validation plus courte

BEST_PATH = "./best_transformer_summarizer.pt"
best_val = float("inf")


def run_epoch_limited(loader, train=True, log_every=50, max_batches=None):
    model.train(train)

    total_loss = 0.0
    total_tokens = 0
    t_start = time.time()
    last_log = t_start

    # nombre réel de batches qu'on va faire
    target_batches = len(loader) if max_batches is None else min(len(loader), max_batches)

    for i, batch in enumerate(loader, start=1):
        if max_batches is not None and i > max_batches:
            break

        src = batch.src_ids.to(device)
        dec = batch.dec_in.to(device)
        y   = batch.labels.to(device)

        logits = model(src, dec)
        loss = label_smoothed_ce(
            logits, y,
            eps=LABEL_SMOOTH,
            ignore_index=PAD
        )

        if train:
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            optimizer.step()

        n_tok = (y != PAD).sum().item()
        total_loss += loss.item() * n_tok
        total_tokens += n_tok

        # LOGGING
        if i == 1:
            print("→ first batch OK")

        if (i % log_every == 0) or (i == target_batches):
            now = time.time()
            dt = now - last_log
            elapsed = now - t_start

            avg_loss = total_loss / max(1, total_tokens)
            # vitesse basée sur le nombre de batches depuis le dernier log
            steps_since_last = log_every if i % log_every == 0 else (i % log_every)
            speed = steps_since_last / max(dt, 1e-6)

            pct = 100 * i / target_batches
            eta = elapsed * (target_batches / i - 1)

            print(
                f"[{'TRAIN' if train else 'VAL'}] "
                f"batch {i:5d}/{target_batches} | "
                f"{pct:5.1f}% | "
                f"loss {avg_loss:.4f} | "
                f"{speed:.2f} batches/s | "
                f"ETA {eta/60:.1f} min"
            )

            last_log = now

    return total_loss / max(1, total_tokens)

## Lancement de l'entraînement avec logging détaillé

On lance l'entraînement epoch par epoch en utilisant `run_epoch`.
Le logging affiche :
- confirmation du premier batch
- progression (%)
- loss moyenne
- vitesse (batches/s)
- ETA estimée

Objectif : vérifier que l'entraînement avance et diagnostiquer les performances.


In [38]:
print(" Début entraînement (batch limitées)")
print("Device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

for epoch in range(1, EPOCHS + 1):
    print("\n" + "="*90)
    print(f"Epoch {epoch}/{EPOCHS} (limité à {MAX_TRAIN_BATCHES} batches train, {MAX_VAL_BATCHES} val)")

    train_loss = run_epoch_limited(
        train_loader,
        train=True,
        log_every=50,
        max_batches=MAX_TRAIN_BATCHES
    )

    val_loss = run_epoch_limited(
        val_loader,
        train=False,
        log_every=50,
        max_batches=MAX_VAL_BATCHES
    )

    print(f"\n Résumé Epoch {epoch} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), BEST_PATH)
        print(f" Nouveau meilleur modèle sauvegardé: {BEST_PATH}")

print("\n Entraînement terminé")


 Début entraînement (batch limitées)
Device: cuda
GPU: NVIDIA GeForce RTX 2070 Super with Max-Q Design

Epoch 1/12 (limité à 1000 batches train, 300 val)
→ first batch OK
[TRAIN] batch    50/1000 |   5.0% | loss 7.7284 | 3.00 batches/s | ETA 5.3 min
[TRAIN] batch   100/1000 |  10.0% | loss 7.3124 | 3.39 batches/s | ETA 4.7 min
[TRAIN] batch   150/1000 |  15.0% | loss 7.1098 | 4.18 batches/s | ETA 4.1 min
[TRAIN] batch   200/1000 |  20.0% | loss 6.9633 | 4.16 batches/s | ETA 3.7 min
[TRAIN] batch   250/1000 |  25.0% | loss 6.8582 | 4.40 batches/s | ETA 3.3 min
[TRAIN] batch   300/1000 |  30.0% | loss 6.7763 | 4.35 batches/s | ETA 3.0 min
[TRAIN] batch   350/1000 |  35.0% | loss 6.7111 | 4.37 batches/s | ETA 2.8 min
[TRAIN] batch   400/1000 |  40.0% | loss 6.6559 | 4.28 batches/s | ETA 2.5 min
[TRAIN] batch   450/1000 |  45.0% | loss 6.6137 | 4.12 batches/s | ETA 2.3 min
[TRAIN] batch   500/1000 |  50.0% | loss 6.5683 | 4.11 batches/s | ETA 2.1 min
[TRAIN] batch   550/1000 |  55.0% | los

## Génération (greedy) + décodage en texte


On génère un résumé token par token (greedy = argmax).
Pourquoi : simple, rapide, parfait pour vérifier que le modèle “parle” avant d'ajouter beam search.

In [39]:
@torch.no_grad()
def generate_greedy(src_ids, max_new_tokens=MAX_OUT_TOKENS):
    model.eval()
    src_ids = src_ids.to(device)

    B = src_ids.size(0)
    dec = torch.full((B, 1), BOS, dtype=torch.long, device=device)

    for _ in range(max_new_tokens):
        logits = model(src_ids, dec)
        next_tok = logits[:, -1, :].argmax(dim=-1, keepdim=True)
        dec = torch.cat([dec, next_tok], dim=1)
        if (next_tok.squeeze(-1) == EOS).all():
            break
    return dec

def decode_tokens(ids):
    out = []
    for t in ids:
        if t in (PAD, BOS): 
            continue
        if t == EOS:
            break
        out.append(t)
    return sp.decode_ids(out)

# charger le meilleur
model.load_state_dict(torch.load(BEST_PATH, map_location=device))

C:\Users\aurel\AppData\Local\Temp\ipykernel_23204\980151232.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(BEST_PATH, map_location=dev

<All keys matched successfully>

## Évaluation ROUGE (rapide)


ROUGE mesure le recouvrement n-gram entre résumé généré et référence.
On commence sur un nombre limité de batches pour aller vite.

In [40]:
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=False)

@torch.no_grad()
def eval_rouge(loader, n_batches=50):
    model.eval()
    acc = {"rouge1": [], "rouge2": [], "rougeL": []}

    for i, batch in enumerate(loader):
        print(f"Evaluating batch {i+1}/{n_batches}", end="\r")
        if i >= n_batches:
            break
        src = batch.src_ids.to(device)
        gen = generate_greedy(src).cpu().numpy()
        ref = batch.labels.numpy()

        for b in range(gen.shape[0]):
            pred_txt = decode_tokens(gen[b].tolist())
            ref_txt  = decode_tokens(ref[b].tolist())
            s = scorer.score(ref_txt, pred_txt)
            for k in acc:
                acc[k].append(s[k].fmeasure)

    return {k: float(np.mean(v)) for k,v in acc.items()}

eval_rouge(test_loader, n_batches=50)


{'rouge1': 0.16198808296126962,
 'rouge2': 0.0246171041378079,
 'rougeL': 0.12687686719087116}

## Démo qualitative (3 exemples)



En soutenance, tu montres : titre, début du texte, résumé référence, résumé généré.
Pourquoi : ça rend l'évaluation “humaine” et visible (au-delà des métriques).

In [41]:
import random

@torch.no_grad()
def show_examples(df, n=3):
    model.eval()
    idxs = random.sample(range(len(df)), n)

    for i in idxs:
        row = df.iloc[i]
        src_str = make_src(row)

        src_ids = [BOS] + sp.encode_as_ids(src_str)[:MAX_IN_TOKENS-2] + [EOS]
        src_ids = torch.tensor([pad_to(src_ids, MAX_IN_TOKENS)], dtype=torch.long)

        gen_ids = generate_greedy(src_ids)[0].tolist()
        pred = decode_tokens(gen_ids)

        print("="*110)
        if "title" in df.columns:
            print("TITLE:", row["title"])
        print("\nTEXT (début):", str(row["text"])[:600], "...")
        print("\nREF SUMMARY:", str(row["summary"]))
        print("\nPRED SUMMARY:", pred)

show_examples(test_df, n=3)


TITLE: Présidentielle en Tunisie : les trois inconnues du second tour

TEXT (début): A Tunis, le 18 septembre 2019. Zoubeir Souissi / REUTERS La Tunisie aborde dans l’incertitude le second tour de l’élection présidentielle, après le premier tour du dimanche 15 septembre, qui a placé en tête le juriste conservateur Kaïs Saïed (18,40 %), devant le magnat de la télévision Nabil Karoui (15,58 %). Bien des hypothèques demeurent sur la suite de ce processus électoral crucial pour la consolidation de la transition démocratique en Tunisie, berceau et seul rescapé de la vague des « printemps arabes ». Quand aura lieu le second tour ? Le tribunal administratif de Tunis a rejeté lundi 23 ...

REF SUMMARY: Calendrier du scrutin, maintien en détention du candidat Nabil Karoui, influence des législatives… De nombreuses incertitudes marquent la campagne d’entre-deux-tours.

PRED SUMMARY: Le président de la République a annoncé jeudi que le président de la République a été mis en examen pour « « incit